# 単純なインタフェースにはクラスの代わりに関数を使う

In [ ]:
# 関数を渡すことによって振る舞いをカスタマイズできる組み込み API が多くあり
# そのような仕組みをフックと呼ぶ
names = ['Socrates', 'Archimedes', 'Plato', 'Aristotle']
# key フックとして len 関数を指定
names.sort(key=len)
print(names)

['Plato', 'Socrates', 'Aristotle', 'Archimedes']


In [2]:
def log_missing():
  print('Key added')
  return 0

In [ ]:
from collections import defaultdict

current = {'green': 12, 'blue': 3}
increments = [
  ('red', 5),
  ('blue', 17),
  ('orange', 9)
]
# 存在しないキーにアクセスしたら、log_missing() を呼び出して初期値(0)を入れる
# current をもとに作っているので、result は {'green': 12, 'blue': 3} を最初に持っている
result = defaultdict(log_missing, current)
print('Before: ', dict(result))
for key, amount in increments:
  result[key] += amount
print('After: ', dict(result))

Before:  {'green': 12, 'blue': 3}
Key added
Key added
After:  {'green': 12, 'blue': 20, 'red': 5, 'orange': 9}


1. key = 'red', amount = 5
   - 'red' はまだ存在しないキー。
   - → result['red'] += 5 と書かれているので：
      1. result['red'] をまず参照 → 無いので log_missing() が呼ばれる。
      1. Key added が表示され、値 0 が返ってくる。
      1. result['red'] = 0 + 5 → 5 となる。
   - 'red': 5 が辞書に追加される。
1. key = 'blue', amount = 17
   - 'blue' はすでに存在（値3）。
   - → result['blue'] += 17 → 3 + 17 = 20
   - 'blue': 20 に更新される。
   - ※ log_missing() は呼ばれない！
1. key = 'orange', amount = 9
   - 'orange' は存在しないキー。
   - → log_missing() が呼ばれ、0 が返される。
   - → result['orange'] = 0 + 9 → 9 になる。

In [ ]:
# defaultdict は missing というフックが状態を保持していることをまったく関知しないにも関わらず、
# 期待された結果の 2 が得られる
def increment_with_report(current, increments):
  added_count = 0

  def missing():
    # ステートフルクロージャ
    # add_count が外側のスコープで定義されていることを宣言
    nonlocal added_count
    added_count += 1
    return 0
  
  result = defaultdict(missing, current)
  for key, amount in increments:
    result[key] += amount

  return result, added_count

In [7]:
result, count = increment_with_report(current, increments)
assert count == 2

In [9]:
# クロージャが状態を持つフックとすることの問題は、状態を持たない関数の例に比べて、読みにくいこと
# 別の方法として、追跡したい状態をカプセル化した軽量なクラスを定義する方法もある
class CountMissing:
  def __init__(self):
    self.added = 0
  
  def missing(self):
    self.added += 1
    return 0

In [ ]:
# ヘルパークラスを使って、状態を持つクロージャの振る舞いを提供することは、
# 先ほどの increment_with_report 関数よりもコードが明瞭になる
counter = CountMissing()
result = defaultdict(counter.missing, current) # メソッド参照
for key, amount in increments:
  result[key] += amount
assert counter.added == 2

In [11]:
# しかし、CountMissing クラス単独で見ると、このクラスの目的が何であるかがすぐにはわからない
# 誰が CountMissing オブジェクトを作るのか、誰が missing メソッドを呼ぶのか
# このような状況を切り抜けるために、Python はクラスで特殊メソッド __call__ を定義できる
class BetterCountMissing:
  def __init__(self):
    self.added = 0

  def __call__(self):
    self.added += 1
    return 0
  
counter = BetterCountMissing()
assert counter() == 0
assert callable(counter)

In [ ]:
# CountMissing.missing の例よりもずっとわかりやすくなっている
# __call__ メソッドは、クラスのインスタンスがどこかで（API フックのように）
# 関数引数として使われてもよいことを示唆している
counter = BetterCountMissing()
result = defaultdict(counter, current) # __call__ を信頼する
for key, amount in increments:
  result[key] += amount
assert counter.added == 2

__call__ を使っても、何が起こっているかについて defaultdict が何も知らなくてもよい
defaultdict に必要なことは、デフォルト値をフックする関数だけである